<a href="https://colab.research.google.com/github/FelipeLuengo85/Fork/blob/main/solucion_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
from datetime import datetime
import math
import sqlite3
import os



# Creación de Directorio a Discresión

os.makedirs('C/sqlite', exist_ok=True)

# verifica existencia de path
if os.path.exists('C/sqlite/DB_consola.sqlite3'):
  # Creación de Conexión o Base de Datos
  print('modelo ya existente')
else:
  con = sqlite3.connect('C/sqlite/DB_consola.sqlite3')

  sql_create_usuario_table = """ CREATE TABLE IF NOT EXISTS tbl_usuario (
                                        id_usuario integer PRIMARY KEY AUTOINCREMENT,
                                        is_name varchar(255) NOT NULL,
                                        is_estado boolean NOT NULL,
                                        dt_fecha_modificacion datetime,
                                        dt_fecha_creacion datetime,
                                        id_external varchar(255),
                                        is_activo boolean NOT NULL
                                    ); """

  sql_create_ejecucion_table = """ CREATE TABLE IF NOT EXISTS tbl_ejecucion (
                                        id_ejecucion integer PRIMARY KEY AUTOINCREMENT,
                                        is_estado_ejecucion boolean NOT NULL,
                                        id_usuario_modificacion integer,
                                        dt_fecha_modificacion datetime,
                                        id_usuario_creacion integer NOT NULL,
                                        dt_fecha_creacion datetime NOT NULL,
                                        is_activo boolean NOT NULL,
                                        FOREIGN KEY (id_usuario_modificacion) REFERENCES tbl_usuario (id_usuario),
                                        FOREIGN KEY (id_usuario_creacion) REFERENCES tbl_usuario (id_usuario)
                                    ); """

  sql_create_company_table = """ CREATE TABLE IF NOT EXISTS tbl_company (
                                        id_company integer PRIMARY KEY AUTOINCREMENT,
                                        is_name varchar(255) NOT NULL,
                                        id_usuario_modificacion integer,
                                        dt_fecha_modificacion datetime,
                                        id_usuario_creacion integer NOT NULL,
                                        dt_fecha_creacion datetime NOT NULL,
                                        id_external varchar(255),
                                        is_activo boolean NOT NULL,
                                        FOREIGN KEY (id_usuario_modificacion) REFERENCES tbl_usuario (id_usuario),
                                        FOREIGN KEY (id_usuario_creacion) REFERENCES tbl_usuario (id_usuario)
                                    ); """

  sql_create_console_table = """ CREATE TABLE IF NOT EXISTS tbl_console (
                                        id_console integer PRIMARY KEY AUTOINCREMENT,
                                        is_name varchar(255) NOT NULL,
                                        id_company interger,
                                        id_usuario_modificacion integer,
                                        dt_fecha_modificacion datetime,
                                        id_usuario_creacion integer NOT NULL,
                                        dt_fecha_creacion datetime NOT NULL,
                                        id_external varchar(255),
                                        is_activo boolean NOT NULL,
                                        FOREIGN KEY (id_usuario_modificacion) REFERENCES tbl_usuario (id_usuario),
                                        FOREIGN KEY (id_usuario_creacion) REFERENCES tbl_usuario (id_usuario),
                                        FOREIGN KEY (id_company) REFERENCES tbl_company (id_company)
                                    ); """

  sql_create_name_game_table = """ CREATE TABLE IF NOT EXISTS tbl_name_game (
                                        id_name_game integer PRIMARY KEY AUTOINCREMENT,
                                        is_name varchar(255) NOT NULL,
                                        id_console integer NOT NULL,
                                        id_usuario_modificacion integer,
                                        dt_fecha_modificacion datetime,
                                        id_usuario_creacion integer NOT NULL,
                                        dt_fecha_creacion datetime NOT NULL,
                                        id_external varchar(255),
                                        is_activo boolean NOT NULL,
                                        FOREIGN KEY (id_usuario_modificacion) REFERENCES tbl_usuario (id_usuario),
                                        FOREIGN KEY (id_usuario_creacion) REFERENCES tbl_usuario (id_usuario),
                                        FOREIGN KEY (id_console) REFERENCES tbl_console (id_console)
                                    ); """

  sql_create_registros_table = """ CREATE TABLE IF NOT EXISTS tbl_registros (
                                        id_registro integer PRIMARY KEY AUTOINCREMENT,
                                        id_name_game integer NOT NULL,
                                        val_fecha_registro datetime,
                                        val_metascore integer,
                                        val_userscore decimal(5,1),
                                        error_console varchar(255),
                                        error_fecha_registro varchar(255),
                                        error_metascore varchar(255),
                                        error_userscore varchar(255),
                                        id_usuario_modificacion integer,
                                        dt_fecha_modificacion datetime,
                                        id_usuario_creacion integer NOT NULL,
                                        dt_fecha_creacion datetime NOT NULL,
                                        id_external varchar(255),
                                        is_activo boolean NOT NULL,
                                        FOREIGN KEY (id_usuario_modificacion) REFERENCES tbl_usuario (id_usuario),
                                        FOREIGN KEY (id_usuario_creacion) REFERENCES tbl_usuario (id_usuario),
                                        FOREIGN KEY (id_name_game) REFERENCES tbl_name_game (id_name_game)
                                    ); """


  cur = con.cursor()
  cur.execute(sql_create_usuario_table)
  cur.execute(sql_create_ejecucion_table)
  cur.execute(sql_create_company_table)
  cur.execute(sql_create_console_table)
  cur.execute(sql_create_name_game_table)
  cur.execute(sql_create_registros_table)


url_1 = 'https://raw.githubusercontent.com/walmartdigital/de-challenge/main/data/result.csv'
df_1 = pd.read_csv(url_1)
large_1 = len(df_1)
url_2 = 'https://raw.githubusercontent.com/walmartdigital/de-challenge/main/data/consoles.csv'
df_2 = pd.read_csv(url_2)
large_2 = len(df_2)

# limpiar encabezados columnas

df_1.columns = df_1.columns.str.strip()
df_2.columns = df_2.columns.str.strip()

# limpiar columnas cruce y campos

df_1['console'] = df_1['console'].str.strip()
df_2['company'] = df_2['company'].str.strip()

df_1['name'] = df_1['name'].str.strip()
df_1['date'] = df_1['date'].str.strip()
df_2['console'] = df_2['console'].str.strip()

# cruce de información entre data frame

df_3 = pd.merge(df_1,df_2,left_on='console', right_on='console',how='outer')

# crear campo para verificacion manejo errores

df_3 = df_3.assign(error_console = math.nan)
df_3 = df_3.assign(error_date = math.nan)
df_3 = df_3.assign(error_metascore = math.nan)
df_3 = df_3.assign(error_userscore = math.nan)


# Verificacion de tipo de dato


df_3['date'] = pd.to_datetime(df_3['date'], errors='coerce')
df_3['metascore'] = pd.to_numeric(df_3['metascore'], errors='coerce')
df_3['userscore'] = pd.to_numeric(df_3['userscore'], errors='coerce')

df_3['error_console'].loc[df_3.index[df_3['name'].isnull() == True]] = 'Error console: tipo valor no valido'
df_3['error_date'].loc[df_3.index[df_3['date'].isnull() == True]] = 'Error date: tipo valor no valido'
df_3['error_metascore'].loc[df_3.index[df_3['metascore'].isnull() == True]] = 'Error metascore: tipo valor no valido'
df_3['error_userscore'].loc[df_3.index[df_3['userscore'].isnull() == True]] = 'Error userscore: tipo valor no valido'

# verificación de información de rango valores aceptables

df_3['error_metascore'].loc[df_3.index[(df_3['metascore'] > 100)]] = 'Error metascore: valor no valido'
df_3['error_metascore'].loc[df_3.index[(df_3['metascore'] < 0)]] = 'Error metascore: valor no valido'

df_3['error_userscore'].loc[df_3.index[(df_3['userscore'] > 10)]] = 'Error userscore: valor no valido'
df_3['error_userscore'].loc[df_3.index[(df_3['userscore'] < 0)]] = 'Error userscore: valor no valido'

#fecha proceso

fecha_proceso = datetime.now()

# incoporar fecha de proceso

df_3 = df_3.assign(fecha_proceso = fecha_proceso)

# insertar datos a tabla source

df_3.to_sql('tbl_source',con)


# verificar dato de usuario generico
# 0 = inactivo ; 1 = activo

df = pd.read_sql_query("select distinct count(*) as datos from tbl_usuario where id_external = 'etl'", con)
if df['datos'].loc[0] == 0:
  sql_insert_usuario_table = """ INSERT INTO tbl_usuario VALUES (null,'etl',1,null,date(),'etl',1); """
  cur.executescript(sql_insert_usuario_table)
  con.commit()

# carga de informacion a tablas

# carga ejecucion

ejecucion = [(fecha_proceso,fecha_proceso)]
cur.executemany("insert into tbl_ejecucion select distinct null, 1, id_usuario, ?, id_usuario, ?, 1 from tbl_usuario where id_external = 'etl' and is_activo = 1",ejecucion)
con.commit()

# carga company

sql_update_company_table = """ UPDATE tbl_company 
                               SET
                               id_usuario_modificacion = (select distinct id_usuario from tbl_usuario where id_external = 'etl' and is_activo = 1),
                               dt_fecha_modificacion = (select distinct fecha_proceso from tbl_source),
                               is_activo = 1
                               where id_external in (select distinct company from tbl_source)
                               ; """

cur.executescript(sql_update_company_table)
con.commit()


sql_insert_company_table = """ INSERT INTO tbl_company 
                               select distinct null,
                               A.company,
                               null,
                               null,
                               (select distinct id_usuario from tbl_usuario where id_external = 'etl' and is_activo = 1) as id_usuario,
                               A.fecha_proceso,
                               A.company,
                               1
                               from tbl_source as A
                               left join tbl_company as B
                               on B.id_external = A.company
                               where B.id_external is null; """

cur.executescript(sql_insert_company_table)
con.commit()


# carga console

sql_update_console_table = """ UPDATE tbl_console 
                               SET
                               id_usuario_modificacion = (select distinct id_usuario from tbl_usuario where id_external = 'etl' and is_activo = 1),
                               dt_fecha_modificacion = (select distinct fecha_proceso from tbl_source),
                               is_activo = 1
                               where id_external in (select distinct console from tbl_source)
                               ; """

cur.executescript(sql_update_console_table)
con.commit()


sql_insert_console_table = """ INSERT INTO tbl_console 
                               select distinct null,
                               A.console,
                               B.id_company,
                               null,
                               null,
                               (select distinct id_usuario from tbl_usuario where id_external = 'etl' and is_activo = 1) as id_usuario,
                               A.fecha_proceso,
                               A.console,
                               1
                               from tbl_source as A
                               inner join tbl_company as B
                               on B.id_external = A.company
                               left join tbl_console as C
                               on A.console = C.id_external
                               where C.id_external is null
                               ; """

cur.executescript(sql_insert_console_table)
con.commit()


# carga name_game

sql_update_name_game_table = """ UPDATE tbl_name_game 
                               SET
                               id_usuario_modificacion = (select distinct id_usuario from tbl_usuario where id_external = 'etl' and is_activo = 1),
                               dt_fecha_modificacion = (select distinct fecha_proceso from tbl_source),
                               is_activo = 1
                               where id_external in (select distinct name from tbl_source)
                               ; """

cur.executescript(sql_update_name_game_table)
con.commit()


sql_insert_name_game_table = """ INSERT INTO tbl_name_game 
                               select distinct null, 
                               A.name,
                               B.id_console,
                               null,
                               null,
                               (select distinct id_usuario from tbl_usuario where id_external = 'etl' and is_activo = 1) as id_usuario,
                               A.fecha_proceso,
                               A.name,
                               1
                               from tbl_source as A
                               inner join tbl_console as B
                               on B.id_external = A.console
                               left join tbl_name_game as C
                               on A.name = C.id_external
                               where C.id_external is null
                               ; """

cur.executescript(sql_insert_name_game_table)
con.commit()


# carga registros


sql_update_registros_table = """ UPDATE tbl_registros
                               SET
                               id_usuario_modificacion = (select distinct id_usuario from tbl_usuario where id_external = 'etl' and is_activo = 1),
                               dt_fecha_modificacion = (select distinct fecha_proceso from tbl_source),
                               is_activo = 1
                               where id_external in (select distinct name||'_'||substr(date,1,10) from tbl_source)
                               ; """

cur.execute(sql_update_registros_table)
con.commit()


sql_insert_registros_table = """ INSERT INTO tbl_registros 
                               select distinct null,
                               B.id_name_game,
                               A.date,
                               A.metascore,
                               A.userscore,
                               A.error_console,
                               A.error_date,
                               A.error_metascore,
                               A.error_userscore,
                               null,
                               null,
                               (select distinct id_usuario from tbl_usuario where id_external = 'etl' and is_activo = 1) as id_usuario,
                               A.fecha_proceso,
                               A.name||'_'||substr(A.date,1,10),
                               1
                               from tbl_source as A
                               inner join tbl_name_game as B
                               on B.id_external = A.name
                               left join tbl_registros as C
                               on A.name||'_'||substr(A.date,1,10) = C.id_external
                               where C.id_external is null
                               ; """

cur.executescript(sql_insert_registros_table)
con.commit()



# eliminar tabla source

sql_eliminar = """ drop table tbl_source ; """
cur.executescript(sql_eliminar).close
con.commit()




cur.close

con.close








modelo ya existente


/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


<function Connection.close>